## Import


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from loguru import logger as lg
from rich import get_console
from rich import print as rprint
from rich.console import Console

# some magic to make rich work in jupyter
# https://github.com/Textualize/rich/issues/3483
# enable it for every cell output with %load_ext rich
console: Console = get_console()
console.is_jupyter = False

In [ ]:
from snap_fit.config.aruco.aruco_board_config import ArucoBoardConfig
from snap_fit.config.aruco.aruco_detector_config import ArucoDetectorConfig
from snap_fit.config.types import EdgePos
from snap_fit.data_models import SegmentId
from snap_fit.params.snap_fit_params import get_snap_fit_params
from snap_fit.params.snap_fit_params import get_snap_fit_paths
from snap_fit.puzzle.sheet_aruco import SheetAruco
from snap_fit.puzzle.sheet_manager import SheetManager


## Params and config


In [ ]:
snap_fit_params = get_snap_fit_params()
rprint(snap_fit_params)

## SegmentId basics


In [ ]:
# Create a SegmentId
seg_id = SegmentId(sheet_id="sheet_01", piece_id=0, edge_pos=EdgePos.LEFT)

rprint(f"SegmentId: {seg_id}")
rprint(f"repr: {repr(seg_id)}")
rprint(f"as_tuple: {seg_id.as_tuple}")

In [ ]:
# Test hashability - can be used in sets and as dict keys
seg_id2 = SegmentId(sheet_id="sheet_01", piece_id=0, edge_pos=EdgePos.RIGHT)
seg_set = {seg_id, seg_id2}
rprint(f"Set: {seg_set}")

seg_dict = {seg_id: "data_for_left", seg_id2: "data_for_right"}
rprint(f"Dict lookup: {seg_dict[seg_id]}")

In [ ]:
# Test serialization
json_str = seg_id.model_dump_json()
rprint(f"JSON: {json_str}")

restored = SegmentId.model_validate_json(json_str)
rprint(f"Restored: {restored}")
rprint(f"Equal: {seg_id == restored}")

## SheetManager with SegmentId

The following cells demonstrate the new `SheetManager` methods.
To run these, you need actual sheets loaded.


In [ ]:
# 1. Configure ArUco Board and Detector
# Using defaults which match the printed board used for 'data/oca'
board_config = ArucoBoardConfig(markers_x=5, markers_y=7)
detector_config = ArucoDetectorConfig(board=board_config)

# 2. Initialize SheetAruco helper
# crop_margin is automatically calculated from the detector configuration
sheet_aruco = SheetAruco(detector_config)

# 3. Define the loader function
# SheetAruco.load_sheet handles loading, rectification, and Sheet creation
aruco_loader = sheet_aruco.load_sheet

# 4. define base folder
paths = get_snap_fit_paths()
data_dir = paths.data_fol / "oca"
lg.info(f"Loading data from {data_dir}")

# 5. instantiate manager and load
manager = SheetManager()
manager.add_sheets(folder_path=data_dir, pattern="*.jpg", loader_func=aruco_loader)

In [ ]:
# Get all segment IDs
all_ids = manager.get_segment_ids_all()
rprint(f"Total segments: {len(all_ids)}")

# Get segments from other pieces
query_id = all_ids[0]
other_ids = manager.get_segment_ids_other_pieces(query_id)
rprint(f"Other pieces: {len(other_ids)}")

# Get actual segment, piece, sheet
segment = manager.get_segment(query_id)
piece = manager.get_piece_by_segment_id(query_id)
sheet = manager.get_sheet_by_segment_id(query_id)